In [1]:
from datetime import datetime
from datetime import timedelta
import os
path = os.getcwd()
import pandas as pd
from kenpompy.utils import login
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# Returns an authenticated browser that can then be used to scrape pages that require authorization.
browser = login('tarik0219@yahoo.com', 'kenporn123')
import kenpompy.FanMatch as fm
import pandas as pd
import pickle

In [2]:
def kenpom(year, month ,day):
    options = webdriver.ChromeOptions() 
    #options.add_argument(r'''user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data''')
    #"user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data" home
    # "user-data-dir=C:\Users\koric1\AppData\Local\Google\Chrome\User Data" work
    #options.add_argument("--start-maximized")
    #self.options.add_argument("--headless")
    #self.options.add_argument("--no-sandbox")
    #self.options.add_argument("--disable-gpu")
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessa
    driver = webdriver.Chrome(chrome_options= options)

    driver.get('https://kenpom.com/')
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="login"]/input[1]').send_keys('tarik0219@yahoo.com')
    driver.find_element_by_xpath('//*[@id="login"]/input[2]').send_keys('kenporn123')
    driver.find_element_by_xpath('//*[@id="login"]/input[3]').click()
    
    date = str(year)+'-'+str("{0:0=2d}".format(month))+'-'+str("{0:0=2d}".format(day))

    
        
    driver.get('https://kenpom.com/archive.php?d='+date)
    time.sleep(10)
    team = []
    off = []
    deff = []
    tempo = []
    dates = []
    for x in range(1,10):
        try:
            for z in range(1,41):
                team_name = str(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[2]/a').text)
                team_off = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[5]').text)
                team_def = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[7]').text)
                team_t = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[9]').text)
                #print(team_name,team_off,team_def,team_t,team_luck)
                team.append(team_name)
                off.append(team_off)
                deff.append(team_def)
                tempo.append(team_t)
                dates.append(date)
        except:
            break
    driver.close()
    d = {'team': team, 'off': off, "def":deff, "tempo":tempo, 'date':date}
    df2 = pd.DataFrame(data=d)
    return df2

In [3]:
def prediciton(year,month,day):
    browser = login('tarik0219@yahoo.com', 'kenporn123')
    import kenpompy.FanMatch as fm
    test = fm.FanMatch(browser, date = str(year)+'-'+str("{0:0=2d}".format(month))+'-'+str("{0:0=2d}".format(day)))
    df = test.fm_df
    df.dropna(subset=['PredictedScore'], inplace = True)
    df['pw'] = df['PredictedScore'].apply(lambda x: x.split('-')[0])
    df['pl'] = df['PredictedScore'].apply(lambda x: x.split('-')[1])
    df['date'] = str(year)+'-'+str("{0:0=2d}".format(month))+'-'+str("{0:0=2d}".format(day))
    df.drop(['Game', 'Location', 'ThrillScore', 'Comeback', 'Excitement',
       'ThrillScoreRank', 'ExcitementRank', 'ComebackRank', 'MVP',
       'Possessions', 'PredictedScore', 'WinProbability',
       'PredictedMOV', 'PredictedLoser', 'OT', 'Loser', 'LoserRank',
       'LoserScore', 'Winner', 'WinnerRank', 'WinnerScore', 'ActualMOV'], axis = 1, inplace = True)
    df['pw'] = df['pw'].astype(int)
    df['pl'] = df['pl'].astype(int)
    df['ps'] = df['pw'] - df['pl']
    return df

In [4]:
date = str(2020)+'-'+str("{0:0=2d}".format(11))+'-'+str("{0:0=2d}".format(6))

In [5]:
date_time_obj = datetime.strptime(date, '%Y-%m-%d')

In [6]:
date_time_obj + timedelta(days = 1)

datetime.datetime(2020, 11, 7, 0, 0)

In [7]:
date

'2020-11-06'

In [13]:
train = pd.read_csv('18-19.csv')

In [14]:
train

,home,away,home_score,away_score,date,line,score,cover
0,Virginia,Towson,73.0,42.0,2018-11-06,25.0,31.0,1
1,Boston College,Milwaukee,73.0,53.0,2018-11-06,17.0,20.0,1
2,Pittsburgh,Youngstown St.,69.0,53.0,2018-11-06,14.0,16.0,1
3,Notre Dame,Illinois Chicago,84.0,67.0,2018-11-06,16.0,17.0,1
4,Western Michigan,Detroit,89.0,76.0,2018-11-06,12.5,13.0,1
...,...,...,...,...,...,...,...,...
2711,Marshall,Green Bay,90.0,70.0,2019-04-04,5.5,20.0,1
2712,DePaul,South Florida,65.0,77.0,2019-04-05,2.0,-12.0,0
2713,Michigan St.,Texas Tech,51.0,61.0,2019-04-06,2.5,-10.0,0
2714,Virginia,Auburn,63.0,62.0,2019-04-06,5.0,1.0,0


In [43]:
final = pd.DataFrame( columns=columns)

In [44]:
final

,home,away,date,line,score,cover,off_x,def_x,tempo_x,off_y,def_y,tempo_y,prediction


In [48]:
year = 2018
day = 25
month = 12

while(month != 4 or day != 9):
    train = pd.read_csv('18-19.csv')
    date = str(year)+'-'+str("{0:0=2d}".format(month))+'-'+str("{0:0=2d}".format(day))
    date_time_obj = datetime.strptime(date, '%Y-%m-%d')
    
    kp = kenpom(year, month ,day)
    train = pd.merge(train, kp, how='left', left_on=['home','date'], right_on=['team','date'])
    train = pd.merge(train, kp, how='left', left_on=['away','date'], right_on=['team','date'])
    
    pkp = prediciton(year,month,day)
    
    train = pd.merge(train, pkp, how='left', left_on=['home','date'], right_on=['PredictedWinner','date'])
    train = pd.merge(train, pkp, how='left', left_on=['away','date'], right_on=['PredictedWinner','date'])
    
    train['prediction'] = train.apply(lambda x: x['ps_x'] if x['home'] == x['PredictedWinner_x'] else -x['ps_y'], axis = 1)
    train.dropna(subset=['prediction'], inplace = True)
    train.drop(['home_score', 'away_score','PredictedWinner_x', 'pw_x', 'pl_x', 'ps_x',
       'PredictedWinner_y', 'pw_y', 'pl_y', 'ps_y','team_x','team_y'], axis = 1, inplace = True)
    
    final = pd.concat([final, train])
    
    print(year,month,day)
    tmrw = date_time_obj + timedelta(days = 1)
    year = tmrw.year
    month = tmrw.month
    day = tmrw.day
    
    


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


2018 12 25


IndexError: list index out of range

In [46]:
final

,home,away,date,line,score,cover,off_x,def_x,tempo_x,off_y,def_y,tempo_y,prediction
0,Virginia,Towson,2018-11-06,25.0,31.0,1,112.5,86.9,66.5,96.8,101.8,72.6,24.0
1,Boston College,Milwaukee,2018-11-06,17.0,20.0,1,106.4,95.6,74.4,95.7,103.0,71.5,17.0
2,Pittsburgh,Youngstown St.,2018-11-06,14.0,16.0,1,101.3,98.9,70.5,94.0,108.0,75.3,15.0
3,Notre Dame,Illinois Chicago,2018-11-06,16.0,17.0,1,107.9,96.4,71.2,98.5,100.6,76.5,14.0
4,Western Michigan,Detroit,2018-11-06,12.5,13.0,1,99.0,103.1,73.0,94.5,109.2,75.2,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,Georgia Tech,North Carolina,2018-12-09,-8.5,-23.0,0,101.6,92.4,69.8,118.8,94.3,76.4,22.0
794,N.C. State,Virginia,2018-12-09,-9.0,-1.0,1,114.8,97.6,74.5,116.2,89.3,62.7,-16.0
801,Alabama,Mississippi St.,2018-12-09,2.0,4.0,1,106.6,97.2,71.1,112.1,93.9,68.8,1.0
806,UNLV,Nevada,2018-12-09,-10.0,-17.0,0,102.1,99.4,71.8,120.5,94.4,69.9,-15.0


In [23]:
train.dropna(subset=['prediction'], inplace = True)

In [20]:
columns = train.columns

In [21]:
final = pd.DataFrame( columns=columns)


In [22]:
final

,home,away,date,line,score,cover,off_x,def_x,tempo_x,off_y,def_y,tempo_y,prediction


In [25]:
final = pd.concat([final, train])

In [35]:
train[train['date'] == '2018-11-09']

,home,away,home_score,away_score,date,line,score,cover
48,Green Bay,Indiana St.,74.0,78.0,2018-11-09,0.0,-4.0,0
49,Elon,North Carolina,67.0,116.0,2018-11-09,-17.5,-49.0,0
50,Kentucky,Southern Illinois,71.0,59.0,2018-11-09,14.5,12.0,0
51,UNC Wilmington,Stanford,59.0,72.0,2018-11-09,-3.0,-13.0,0
52,East Carolina,James Madison,72.0,73.0,2018-11-09,-2.5,-1.0,1
53,Auburn,Washington,88.0,66.0,2018-11-09,8.5,22.0,1
54,Northern Illinois,Northern Kentucky,85.0,88.0,2018-11-09,-4.0,-3.0,1
55,Penn,Rice,92.0,76.0,2018-11-09,15.0,16.0,1
56,Texas St.,Air Force,67.0,57.0,2018-11-09,5.0,10.0,1
57,Nevada,Pacific,83.0,61.0,2018-11-09,20.0,22.0,1
